# Обзор данных

In [1]:
# Стандартные библиотеки
import wget
import requests
from tqdm import tqdm
import tarfile
import soundfile as sf
import os
import csv
import pandas as pd
import random
import glob
import librosa

# Для запросов к сайтам
import requests
from bs4 import BeautifulSoup

# Для морфологической разметки
from pymorphy2 import MorphAnalyzer
import nltk
from nltk import word_tokenize, pos_tag
from nltk.tag import map_tag
import string

# Для разметки именнованных сущностей
import spacy
from collections import defaultdict, Counter

## Загрука и распаковка датасета

In [2]:
# Скачивание и распаковка
url = "https://www.openslr.org/resources/12/train-clean-100.tar.gz"

# Путь, куда сохранить архив
download_path = "train-clean-100.tar.gz"
extract_folder = "train-clean-100"

if not os.path.exists(download_path):
    # Загружаем файл
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))  # Размер файла в байтах

    with open(download_path, 'wb') as f, tqdm(desc = download_path,
                                              total = total_size,
                                              unit = 'B',
                                              unit_scale = True,
                                              unit_divisor = 1024
                                             ) as bar:
        for chunk in response.iter_content(chunk_size = 1024):
            if chunk:  # Фильтруем keep-alive chunks
                f.write(chunk)
                bar.update(len(chunk))

if os.path.exists(download_path) and not os.path.exists(extract_folder):
    # Распаковка
    with tarfile.open(download_path, 'r:gz') as tar:
        tar.extractall(extract_folder)

## Файлы .flac в .wav 

In [3]:
# Функция для конвертации файлов .flac в .wav
def convert_flac_to_wav(root_dir):
    for root, dirs, files in os.walk(root_dir):
        flac_files = [f for f in files if f.endswith('.flac')]
        
        for flac_file in tqdm(flac_files, desc = f"Обработка {root}"):
            flac_path = os.path.join(root, flac_file)
            wav_path = os.path.join(root, flac_file.replace('.flac', '.wav'))
            
            # Чтение FLAC и сохранение как WAV
            audio, sr = sf.read(flac_path)
            sf.write(wav_path, audio, sr, format='WAV')
            
            # Удаление исходного FLAC после конвертации
            # os.remove(flac_path)

dataset_path = "train-clean-100"  # Путь к корневой папке датасета
convert_flac_to_wav(dataset_path)

Обработка train-clean-100: 0it [00:00, ?it/s]
Обработка train-clean-100\LibriSpeech: 0it [00:00, ?it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100: 0it [00:00, ?it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\103: 0it [00:00, ?it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\103\1240:   0%|          | 0/58 [00:00<?, ?it/s]

Обработка train-clean-100\LibriSpeech\train-clean-100\103\1240: 100%|██████████| 58/58 [00:00<00:00, 65.41it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\103\1241: 100%|██████████| 44/44 [00:00<00:00, 63.80it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\1034: 0it [00:00, ?it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\1034\121119: 100%|██████████| 96/96 [00:01<00:00, 78.17it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\1040: 0it [00:00, ?it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\1040\133433:  53%|█████▎    | 43/81 [00:00<00:00, 70.27it/s]


KeyboardInterrupt: 

## Файлы .txt в .csv

In [2]:
def convert_trans_to_csv(root_dir):
    for root, dirs, files in os.walk(root_dir):
        for file in tqdm(files, desc = f"Обработка {root}"):
            if not file.endswith('.trans.txt'):
                continue

            trans_path = os.path.join(root, file)
            csv_path = os.path.join(root, file.replace('.trans.txt', 'trans.csv'))
            
            with open(trans_path, 'r') as f_in, open(csv_path, 'w', newline='') as f_out:
                writer = csv.writer(f_out)
                writer.writerow(['id', 'text'])  # Заголовки
                
                for line in f_in:
                    line = line.strip()
                    if not line:
                        continue
                    
                    # Разделение ID и текста (первое слово — ID)
                    parts = line.split(' ', 1)
                    if len(parts) < 2:
                        continue  # Пропуск некорректных строк
                    
                    utterance_id, text = parts
                    writer.writerow([utterance_id, text])

dataset_path = "train-clean-100"  # Путь к корневой папке датасета
convert_trans_to_csv(dataset_path)

Обработка train-clean-100: 0it [00:00, ?it/s]
Обработка train-clean-100\LibriSpeech: 100%|██████████| 5/5 [00:00<?, ?it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100: 0it [00:00, ?it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\103: 0it [00:00, ?it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\103\1240: 100%|██████████| 118/118 [00:00<00:00, 39405.09it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\103\1241: 100%|██████████| 90/90 [00:00<00:00, 29990.26it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\1034: 0it [00:00, ?it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\1034\121119: 100%|██████████| 194/194 [00:00<00:00, 97007.03it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\1040: 0it [00:00, ?it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\1040\133433: 100%|██████████| 164/164 [00:00<00:00, 81986.40it/s]
Обработка train-clean-100\LibriSpeech\train-clean-100\1069: 0it [00:00, ?it/s]
Обработка train-

In [149]:
file_path = "train-clean-100/LibriSpeech/SPEAKERS.TXT"
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Skip the header and process the content
header_line = lines[0].strip()
data_lines = [line.strip() for line in lines[1:] if "train-clean-100" in line]

# Extract columns using the header
columns = [col.strip() for col in header_line.split("|")]

# Parse rows into a list of dictionaries
data = []
for line in data_lines:
    values = [val.strip() for val in line.split("|")]
    if len(values) == len(columns):
        row = dict(zip(columns, values))
        data.append(row)

# Create DataFrame and save as CSV
df = pd.DataFrame(data)
csv_output_path = "speakers.csv"
df.to_csv(csv_output_path, index=False)

In [ ]:
df

,ID,READER,MINUTES,SUBSET,PROJ.,BOOK ID,CH. TITLE,PROJECT TITLE
0,198,19,8.42,train-clean-100,219,121,Chapter 01,Northanger Abbey
1,209,198,8.20,train-clean-100,219,121,Chapter 12,Northanger Abbey
2,216,403,6.30,train-clean-100,219,121,Chapter 19,Northanger Abbey
3,218,89,12.78,train-clean-100,219,121,Chapter 21,Northanger Abbey
4,219,89,12.30,train-clean-100,219,121,Chapter 22,Northanger Abbey
...,...,...,...,...,...,...,...,...
580,298545,8838,12.55,train-clean-100,8386,10446,"The Croxley Master - I, part 1",Green Flag and Other Stories of War and Sport
581,298546,8838,12.51,train-clean-100,8386,10446,"The Croxley Master - I, part 2",Green Flag and Other Stories of War and Sport
582,300880,6064,8.82,train-clean-100,8473,44326,Chapter 10,Wanted: A Husband
583,305212,8630,10.50,train-clean-100,8645,35941,How Augustine Went to Carthage and How Patrici...,Life of Saint Monica


In [6]:
data = pd.read_csv('train-clean-100/LibriSpeech/train_clean_100_chapters.csv')
data

,ID,READER,MINUTES,SUBSET,PROJ.,BOOK ID,CH. TITLE,PROJECT TITLE
0,198,19,8.42,train-clean-100,219,121,Chapter 01,Northanger Abbey
1,209,198,8.20,train-clean-100,219,121,Chapter 12,Northanger Abbey
2,216,403,6.30,train-clean-100,219,121,Chapter 19,Northanger Abbey
3,218,89,12.78,train-clean-100,219,121,Chapter 21,Northanger Abbey
4,219,89,12.30,train-clean-100,219,121,Chapter 22,Northanger Abbey
...,...,...,...,...,...,...,...,...
580,298545,8838,12.55,train-clean-100,8386,10446,"The Croxley Master - I, part 1",Green Flag and Other Stories of War and Sport
581,298546,8838,12.51,train-clean-100,8386,10446,"The Croxley Master - I, part 2",Green Flag and Other Stories of War and Sport
582,300880,6064,8.82,train-clean-100,8473,44326,Chapter 10,Wanted: A Husband
583,305212,8630,10.50,train-clean-100,8645,35941,How Augustine Went to Carthage and How Patrici...,Life of Saint Monica


In [ ]:
# Настройки
CSV_PATH = "train-clean-100/LibriSpeech/train_clean_100_chapters.csv"
COVERS_DIR = "book_covers"

# Создание папки для обложек
os.makedirs(COVERS_DIR, exist_ok=True)

def get_gutenberg_cover_url(title):
    """Ищет URL обложки на Project Gutenberg."""
    search_url = f"https://www.gutenberg.org/ebooks/search/?query={title}"
    try:
        response = requests.get(search_url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        book_link = soup.find("a", href=lambda x: x and "/ebooks/" in x)
        if book_link:
            book_id = book_link["href"].split("/")[-1]
            return f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.cover.medium.jpg"
    except Exception as e:
        print(f"[Gutenberg] Ошибка для '{title}': {e}")
    return None

def get_openlibrary_cover_url(title):
    """Ищет URL обложки на Open Library."""
    try:
        url = f"https://openlibrary.org/search.json?q={title}"
        response = requests.get(url, timeout=10).json()
        if response["num_found"] > 0:
            cover_id = response["docs"][0].get("cover_i")
            if cover_id:
                return f"https://covers.openlibrary.org/b/id/{cover_id}-M.jpg"
    except Exception as e:
        print(f"[OpenLibrary] Ошибка для '{title}': {e}")
    return None

def download_cover(book_id, title):
    """Скачивает обложку, если её ещё нет."""
    cover_path = f"{COVERS_DIR}/{book_id}.jpg"
    
    # Если обложка уже существует - пропускаем
    if os.path.exists(cover_path):
        return True
    
    # Порядок проверки источников
    apis = [
        ("OpenLibrary", lambda: get_openlibrary_cover_url(title)),
        ("Gutenberg", lambda: get_gutenberg_cover_url(title)),
    ]
    
    for source, api_func in apis:
        cover_url = api_func()
        if cover_url:
            try:
                response = requests.get(cover_url, stream=True, timeout=10)
                if response.status_code == 200:
                    with open(cover_path, "wb") as f:
                        f.write(response.content)
                    print(f"[Успех] {book_id}: Обложка найдена через {source}")
                    return True
            except Exception as e:
                print(f"[Ошибка] {book_id}: Не удалось скачать с {source} ({cover_url}): {e}")
    
    print(f"[Пропуск] {book_id}: Обложка не найдена ни в одном источнике")
    return False

# Основной процесс
df = pd.read_csv(CSV_PATH)
unique_books = df["BOOK ID"].drop_duplicates()

success_count = 0
for _, row in tqdm(unique_books.iterrows(), total=len(unique_books)):
    if download_cover(row["BOOK ID"], row["PROJECT TITLE"]):
        success_count += 1

print(f"\nИтог: Скачано {success_count}/{len(unique_books)} обложек.")

  0%|          | 1/320 [00:11<59:34, 11.21s/it]

[Успех] 121: Обложка найдена через OpenLibrary


  1%|          | 2/320 [00:13<31:23,  5.92s/it]

[Успех] 84: Обложка найдена через OpenLibrary


  1%|          | 3/320 [00:15<21:38,  4.09s/it]

...

[Успех] 10446: Обложка найдена через OpenLibrary


100%|█████████▉| 319/320 [26:25<00:05,  5.76s/it]

[Успех] 44326: Обложка найдена через Gutenberg


100%|██████████| 320/320 [26:27<00:00,  4.96s/it]

[Успех] 35941: Обложка найдена через Gutenberg

Итог: Скачано 298/320 обложек.


## Загрузка иллюстраций

In [6]:
# Основной процесс
CSV_PATH = "train-clean-100/LibriSpeech/train_clean_100_chapters.csv"
df = pd.read_csv(CSV_PATH)
unique_books = df["PROJECT TITLE"].drop_duplicates().to_list()
unique_books

['Northanger Abbey',
 'Frankenstein, or The Modern Prometheus',
 'Kwaidan: Stories and Studies of Strange Things',
 'Journal of the Plague Year',
 'Anne of Green Gables (version 3)',
 'Brigands of the Moon',
 'Surprising Adventures of Baron Munchausen',
 'Lock and Key Library',
 "Clotel, or, The President's Daughter",
 'For the Term of His Natural Life',
 'Uncle Silas',
 'East Lynne',
 'Just Patty',
 'Anne of Green Gables (version 5)',
 'Works of Edgar Allan Poe, Raven Edition, Volume 2',
 'Cossacks',
 'Autobiography',
 'Legacy',
 'Autobiography of Anthony Trollope',
 'Miserables Vol. 3',
 'Millionaire Baby',
 'Anna Karenina, Book 2',
 'Tanglewood Tales',
 'Theo',
 'Music Notation and Terminology',
 'Once on a Time',
 'Celebrated Crimes, Vol. 1',
 'Gargantua and Pantagruel, Book I',
 'Complete Book of Cheese',
 'Ox-Team Days on the Oregon Trail',
 'Life of Kit Carson',
 'Prime Minister',
 'In His Steps',
 "Sylvia's Lovers",
 'Red Thumb Mark',
 'Jesuits in North America in the 17th Cent

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from urllib.parse import quote_plus

# Настройки
CSV_PATH = "train-clean-100/LibriSpeech/train_clean_100_chapters.csv"
ILLUSTRATIONS_DIR = "book_illustrations"

# Создание папки
os.makedirs(ILLUSTRATIONS_DIR, exist_ok=True)

def get_gutenberg_id_by_title(title):
    """Ищет ID книги по названию на Gutenberg."""
    search_url = f"https://www.gutenberg.org/ebooks/search/?query={quote_plus(title)}"
    try:
        response = requests.get(search_url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        link = soup.find("li", class_="booklink")
        if link:
            href = link.find("a", class_="link")["href"]
            return int(href.strip("/").split("/")[-1])
    except Exception as e:
        print(f"[Поиск ID] Ошибка для '{title}': {e}")
    return None

def download_illustrations(book_id, save_subdir):
    """Скачивает все изображения из HTML-версии книги."""
    html_url = f"https://www.gutenberg.org/files/{book_id}/{book_id}-h/{book_id}-h.htm"
    try:
        response = requests.get(html_url, timeout=10)
        if response.status_code != 200:
            return 0
        soup = BeautifulSoup(response.content, "lxml-xml")
        img_tags = soup.find_all("img")
        
        os.makedirs(save_subdir, exist_ok=True)
        count = 0
        for img in img_tags:
            src = img.get("src")
            if src:
                full_url = html_url.rsplit("/", 1)[0] + "/" + src
                img_name = os.path.basename(src).split("?")[0]
                img_path = os.path.join(save_subdir, img_name)
                try:
                    img_data = requests.get(full_url, timeout=10).content
                    with open(img_path, "wb") as f:
                        f.write(img_data)
                    count += 1
                except Exception as e:
                    print(f"[Скачивание] Ошибка {full_url}: {e}")
        return count
    except Exception as e:
        print(f"[HTML] Ошибка при загрузке HTML для ID {book_id}: {e}")
        return 0

# Основной процесс
df = pd.read_csv(CSV_PATH)
unique_books = df[["BOOK ID", "PROJECT TITLE"]].drop_duplicates()

not_found = []
success_count = 0

for _, row in tqdm(unique_books.iterrows(), total=len(unique_books), desc="Книги"):
    book_id = row["BOOK ID"]
    title = row["PROJECT TITLE"]
    gutenberg_id = get_gutenberg_id_by_title(title)
    
    if gutenberg_id:
        save_path = os.path.join(ILLUSTRATIONS_DIR, str(book_id))
        count = download_illustrations(gutenberg_id, save_path)
        if count > 0:
            print(f"[Успех] {book_id} ({title}): загружено {count} изображений")
            success_count += 1
        else:
            print(f"[Пропуск] {book_id} ({title}): нет изображений")
            not_found.append({"BOOK ID": book_id, "TITLE": title, "REASON": "Нет изображений"})
    else:
        print(f"[Пропуск] {book_id} ({title}): не найден на Gutenberg")
        not_found.append({"BOOK ID": book_id, "TITLE": title, "REASON": "Не найден на Gutenberg"})

# Сохраняем список неудачных загрузок
not_found_df = pd.DataFrame(not_found)
not_found_df.to_csv("not_found_illustrations.csv", index=False)

print(f"\nИтог: Иллюстрации найдены у {success_count} книг из {len(unique_books)}")
# print(f"Список пропущенных сохранён в not_found_illustrations.csv")

Книги:   0%|          | 1/320 [00:02<11:46,  2.22s/it]

[Пропуск] 121 (Northanger Abbey): нет изображений


Книги:   1%|          | 2/320 [00:05<13:49,  2.61s/it]

[Пропуск] 84 (Frankenstein, or The Modern Prometheus): нет изображений


Книги:   1%|          | 3/320 [00:10<20:35,  3.90s/it]

...

[Пропуск] 10446 (Green Flag and Other Stories of War and Sport): нет изображений


Книги: 100%|█████████▉| 319/320 [1:42:12<00:02,  2.96s/it]

[Успех] 44326 (Wanted: A Husband): загружено 2 изображений


Книги: 100%|██████████| 320/320 [1:42:13<00:00, 19.17s/it]

[Пропуск] 35941 (Life of Saint Monica): нет изображений

Итог: Иллюстрации найдены у 135 книг из 320


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm

# Настройки
CSV_PATH = "train-clean-100/LibriSpeech/train_clean_100_chapters.csv"
COVERS_DIR = "book_covers"
false_cover = []
# Создание папки-родителя
os.makedirs(COVERS_DIR, exist_ok=True)

def get_gutenberg_cover_url(title):
    search_url = f"https://www.gutenberg.org/ebooks/search/?query={title}"
    try:
        response = requests.get(search_url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        book_link = soup.find("a", href=lambda x: x and "/ebooks/" in x)
        if book_link:
            book_id = book_link["href"].split("/")[-1]
            return f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.cover.medium.jpg"
    except Exception as e:
        print(f"[Gutenberg] Ошибка для '{title}': {e}")
    return None

def get_openlibrary_cover_url(title):
    try:
        url = f"https://openlibrary.org/search.json?q={title}"
        response = requests.get(url, timeout=10).json()
        if response["num_found"] > 0:
            cover_id = response["docs"][0].get("cover_i")
            if cover_id:
                return f"https://covers.openlibrary.org/b/id/{cover_id}-M.jpg"
    except Exception as e:
        print(f"[OpenLibrary] Ошибка для '{title}': {e}")
    return None

def download_cover(book_id, title):
    """Скачивает обложку и сохраняет как cover.jpg в папке по book_id."""
    book_dir = os.path.join(COVERS_DIR, str(book_id))
    cover_path = os.path.join(book_dir, "cover.jpg")

    if os.path.exists(cover_path):
        return True

    os.makedirs(book_dir, exist_ok=True)

    apis = [
        ("OpenLibrary", lambda: get_openlibrary_cover_url(title)),
        ("Gutenberg", lambda: get_gutenberg_cover_url(title)),
    ]

    for source, api_func in apis:
        cover_url = api_func()
        if cover_url:
            try:
                response = requests.get(cover_url, stream=True, timeout=10)
                if response.status_code == 200:
                    with open(cover_path, "wb") as f:
                        f.write(response.content)
                    print(f"[Успех] {book_id}: Обложка сохранена из {source}")
                    return True
            except Exception as e:
                print(f"[Ошибка] {book_id}: Скачивание из {source} ({cover_url}): {e}")

    print(f"[Пропуск] {book_id}: Обложка не найдена ни в одном источнике")
    false_cover.append(book_id)
    return False

# Основной процесс
df = pd.read_csv(CSV_PATH)
unique_books = df[["BOOK ID", "PROJECT TITLE"]].drop_duplicates()

success_count = 0
for _, row in tqdm(unique_books.iterrows(), total=len(unique_books), desc="Загрузка обложек"):
    if download_cover(row["BOOK ID"], row["PROJECT TITLE"]):
        success_count += 1

print(f"\nИтог: Скачано {success_count}/{len(unique_books)} обложек.")

Загрузка обложек:  57%|█████▊    | 69/120 [00:01<00:01, 36.64it/s]

[Пропуск] 10030: Обложка не найдена ни в одном источнике


Загрузка обложек: 100%|██████████| 120/120 [00:03<00:00, 32.77it/s]

[Пропуск] 29607: Обложка не найдена ни в одном источнике

Итог: Скачано 118/120 обложек.


## Добавление жанра

In [ ]:
# Загрузка данных
df = pd.read_csv("train-clean-100/LibriSpeech/train_clean_100_chapters.csv")

# Убедимся, что столбец 'ID' содержит уникальные значения
book_ids = df["BOOK ID"]

# Словарь для кэширования результатов
id_to_genre = {}

# Функция запроса жанра по ID
def get_genre(book_id):
    url = f"https://gutendex.com/books/{book_id}/"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            subjects = data.get("subjects", [])
            return "; ".join(subjects) if subjects else "Unknown"
        else:
            return "Not Found"
    except Exception:
        return "Error"

# Обработка с прогресс-баром
for book_id in tqdm(book_ids, desc="Получение жанров"):
    genre = get_genre(book_id)
    id_to_genre[book_id] = genre
    time.sleep(1)  # Пауза в 1 секунду для уважения к API

# Присвоение жанров в DataFrame
df["Genre"] = df["BOOK ID"].map(id_to_genre)

# Сохранение результата
df.to_csv("train_with_genre.csv", index = False)

Получение жанров: 100%|██████████| 585/585 [15:08<00:00,  1.55s/it]


In [8]:
import pandas as pd
import requests
import time
from tqdm import tqdm

# Загрузка данных
df = pd.read_csv("train-clean-100/LibriSpeech/train_clean_100_chapters.csv")

# Получение уникальных ID
book_ids = df["BOOK ID"]

# Подготовка
id_to_genre = {}

# Основная функция поиска жанра
def get_genre_openlibrary(book_id, title):
    # 1. Попробовать найти по Gutenberg ID
    id_url = f"https://openlibrary.org/search.json?gutenberg_id={book_id}"
    try:
        response = requests.get(id_url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data["numFound"] > 0:
                subjects = data["docs"][0].get("subject", [])
                if subjects:
                    return "; ".join(subjects)
    except Exception:
        pass

    # 2. Если не найдено — искать по названию
    title_url = f"https://openlibrary.org/search.json?title={requests.utils.quote(title)}"
    try:
        response = requests.get(title_url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data["numFound"] > 0:
                subjects = data["docs"][0].get("subject", [])
                if subjects:
                    return "; ".join(subjects)
                else:
                    return "Unknown"
            else:
                return "Not Found"
    except Exception:
        return "Error"

# Прогресс по строкам
for _, row in tqdm(df.iterrows(), total=len(df), desc="Поиск жанров OpenLibrary"):
    book_id = row["BOOK ID"]
    title = row["PROJECT TITLE"]
    genre = get_genre_openlibrary(book_id, title)
    id_to_genre[book_id] = genre
    time.sleep(1)  # Пауза между запросами

# Добавление результатов в DataFrame
df["Genre"] = df["BOOK ID"].map(id_to_genre)

# Сохранение
df.to_csv("train_with_genre_openlibrary.csv", index=False)

Поиск жанров OpenLibrary:   0%|          | 0/585 [00:00<?, ?it/s]

Поиск жанров OpenLibrary: 100%|██████████| 585/585 [28:53<00:00,  2.96s/it]


## Объединение всех csv в один

In [17]:
def combine_csv_files(root_dir, output_path='all_POS_NER.csv'):
    all_dfs = []
    
    for subdir, _, files in tqdm(os.walk(root_dir)):
        for file in files:
            if file.endswith('.csv'):
                csv_path = os.path.join(subdir, file)
                try:
                    df = pd.read_csv(csv_path)
                    df['source_file'] = csv_path  # Добавим имя файла как метаинформацию
                    all_dfs.append(df)
                except Exception as e:
                    print(f"Ошибка при чтении {csv_path}: {e}")
    
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        combined_df.to_csv(output_path, index=False)
        print(f"Объединённый файл сохранён как: {output_path}")
    else:
        print("Не найдено CSV-файлов для объединения.")

# Запуска
combine_csv_files('train-clean-100')

839it [00:00, 1198.39it/s]


Объединённый файл сохранён как: all_POS_NER.csv


## Восстановление регистра

In [42]:
import pandas as pd
from typing import List
from punctuators.models import PunctCapSegModelONNX

# Загружаем модель один раз
model: PunctCapSegModelONNX = PunctCapSegModelONNX.from_pretrained("1-800-BAD-CODE/xlm-roberta_punctuation_fullstop_truecase")

# Загружаем датафрейм
df = pd.read_csv("all_POS_NER_split.csv")

def restore_texts(texts: List[str]) -> List[str]:
    # infer принимает список текстов и возвращает список списков (вариантов восстановления)
    results = model.infer(texts = texts, apply_sbd = True)
    # Возьмем первый вариант для каждого текста
    restored = [out[0] if out else txt for out, txt in zip(results, texts)]
    return restored

# Обработка батчами с прогрессбаром tqdm
batch_size = 16
restored_texts = []

for i in tqdm(range(0, len(df), batch_size), desc="Restoring texts"):
    batch_texts = df['text'].iloc[i:i+batch_size].str.lower().tolist()
    batch_restored = restore_texts(batch_texts)
    restored_texts.extend(batch_restored)

df['text'] = restored_texts

# Сохраняем результат
df.to_csv("restored_text.csv", index = False)

d:\Python310\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
Restoring texts: 100%|██████████| 1784/1784 [44:52<00:00,  1.51s/it]


In [60]:
df['text'] = df['text'].str.replace(f'[{string.punctuation}]', '', regex=True)
df.to_csv('restored_text.csv')

## Разметка именнованных сущностей

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Функция для разметки по именнованным сущностям
def ner_tag_text(text):
    doc = nlp(text)
    return " ".join([f"{ent.text}_{ent.label_}" for ent in doc.ents])

# Функция применения разметки ко всем строкам в датафрейме с транскрипциями
def process_csv(csv_path):
    df = pd.read_csv(csv_path)
    if 'text' in df.columns:
        tqdm.pandas(desc=f'POS для {os.path.basename(csv_path)}')
        df['ner_tags'] = df['text'].progress_apply(ner_tag_text)
        df.to_csv(csv_path, index=False)
    else:
        print(f"Пропущено (нет 'text'): {csv_path}")

# Запуск
process_csv('restored_text.csv')

POS для restored_text.csv: 100%|██████████| 28539/28539 [03:49<00:00, 124.15it/s]


## Разметка по морфологии

In [53]:
nltk.download('punkt', force = True)                           # Токенизатор
nltk.download('averaged_perceptron_tagger', force = True)      # POS-теггер
nltk.download('averaged_perceptron_tagger_eng', force = True)
nltk.download('universal_tagset', force = True)                # Универсальные теги

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HUAWEI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HUAWEI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\HUAWEI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\HUAWEI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

In [ ]:
# Функция для разметки по частям речи
def pos_tag_text(text):
    words = word_tokenize(text)  # Токенизация
    tags = pos_tag(words)        # Получаем (слово, тег)
    # Конвертируем теги в универсальный набор и объединяем с словами через "_"
    tagged_words = [f"{word}_{map_tag('en-ptb', 'universal', tag)}" for word, tag in tags]
    return ' '.join(tagged_words)  # Возвращаем строку с объединёнными словами

# Функция применения разметки ко всем строкам в датафрейме с транскрипциями
def process_csv(csv_path):
    df = pd.read_csv(csv_path)
    if 'text' in df.columns:
        tqdm.pandas(desc=f'POS для {os.path.basename(csv_path)}')
        df['pos_tags'] = df['text'].progress_apply(pos_tag_text)
        df.to_csv(csv_path, index=False)
    else:
        print(f"Пропущено (нет 'text'): {csv_path}")

# Запуск
process_csv('restored_text.csv')

POS для restored_text.csv: 100%|██████████| 28539/28539 [00:21<00:00, 1342.83it/s]


## Разметка по атмосфере

In [72]:
import pandas as pd
from collections import defaultdict

# 1. Загрузка NRC Lexicon (предполагаем формат: слово<TAB>категория<TAB>оценка)
nrc_lexicon_path = "NRC-Emotion-Lexicon/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
nrc_data = pd.read_csv(nrc_lexicon_path, sep='\t', header=None, names=['word', 'emotion', 'score'])
nrc_data

,word,emotion,score
0,aback,anger,0
1,aback,anticipation,0
2,aback,disgust,0
3,aback,fear,0
4,aback,joy,0
...,...,...,...
141535,zoom,negative,0
141536,zoom,positive,0
141537,zoom,sadness,0
141538,zoom,surprise,0


In [116]:
atmosphere_by_emotion = {
    "anger": ["hostile", "aggressive", "tense", "furious", "stormy"],
    "anticipation": ["eager", "restless", "curious", "hopeful", "watchful"],
    "disgust": ["repulsive", "disturbing", "grim", "nauseating"],
    "fear": ["terrifying", "eerie", "paranoid", "claustrophobic", "ominous"],
    "joy": ["joyful", "bright", "cheerful", "delightful", "vibrant"],
    "sadness": ["melancholic", "gloomy", "somber", "tearful", "mournful"],
    "surprise": ["shocking", "unexpected", "abrupt", "dramatic", "puzzling"],
    "trust": ["calm", "secure", "warm", "gentle", "reassuring"],
    "positive": ["peaceful", "sunny", "harmonious", "pleasant", "inspiring"],
    "negative": ["bleak", "harsh", "cold", "dark", "depressing"]
}

# Создаем словарь {слово: [список эмоций]}
word_to_emotions = defaultdict(list)
for _, row in nrc_data.iterrows():
    if row['score'] == 1:
        word_to_emotions[row['word']].append(row['emotion'])

# Функция для разметки текста
def tag_text(text):
    words = text.lower().split()
    tagged_words = []
    emotions_in_text = []
    
    for word in words:
        if word in word_to_emotions:
            # Берем первую эмоцию (или можно выбрать случайную/наиболее вероятную)
            emotion = word_to_emotions[word][0]  
            tagged_words.append(f"{word}_{emotion}")
            emotions_in_text.append(emotion)
    
    # Определяем общую атмосферу (самая частая эмоция)
    overall_mood = Counter(emotions_in_text).most_common(1)
    overall_mood = overall_mood[0][0] if overall_mood else "neutral"

    # Случайный атмосферный тег из словаря
    atmosphere = random.choice(atmosphere_by_emotion.get(overall_mood, ["neutral"]))
    
    return " ".join(tagged_words), atmosphere

In [117]:
# Применяем функцию к DataFrame
df[["emotion_text", "overall_mood"]] = df["text"].apply(lambda x: pd.Series(tag_text(x)))
df

,reader,book_id,sent_id,text,pos_tags,ner_tags,source_file,emotion_text,overall_mood
0,103,1240,0,CHAPTER One Missus Rachel Lynde is surprised M...,CHAPTER_NOUN One_NUM Missus_NOUN Rachel_NOUN L...,CHAPTER_ORG Missus Rachel Lynde_PERSON Missus ...,train-clean-100\LibriSpeech\train-clean-100\10...,surprised_surprise main_positive hollow_negative,unexpected
1,103,1240,1,That had its source away,That_DET had_VERB its_PRON source_NOUN away_ADV,NaN,train-clean-100\LibriSpeech\train-clean-100\10...,,neutral
2,103,1240,2,For not even a brook could run past Missus Rac...,For_ADP not_ADV even_ADV a_DET brook_NOUN coul...,Missus Rachel Lyndes_PERSON,train-clean-100\LibriSpeech\train-clean-100\10...,decency_positive,sunny
3,103,1240,3,And that if she noticed anything odd or out of...,And_CONJ that_ADP if_ADP she_PRON noticed_VERB...,NaN,train-clean-100\LibriSpeech\train-clean-100\10...,rest_positive,harmonious
4,103,1240,4,But Missus Rachel Lynde was one of those capab...,But_CONJ Missus_NOUN Rachel_NOUN Lynde_NOUN wa...,Missus Rachel Lynde_PERSON,train-clean-100\LibriSpeech\train-clean-100\10...,manage_positive bargain_positive,harmonious
...,...,...,...,...,...,...,...,...,...
28534,911,130578,16,He put an arrow through it and it turned a som...,He_PRON put_VERB an_DET arrow_NOUN through_ADP...,NaN,train-clean-100\LibriSpeech\train-clean-100\91...,fell_negative,depressing
28535,911,130578,17,And ran away,And_CONJ ran_VERB away_ADV,NaN,train-clean-100\LibriSpeech\train-clean-100\91...,,neutral
28536,911,130578,18,A man suddenly stood before him and asked what...,A_DET man_NOUN suddenly_ADV stood_VERB before_...,NaN,train-clean-100\LibriSpeech\train-clean-100\91...,suddenly_surprise cry_negative,dramatic
28537,911,130578,19,And then whip the body with a Meral twig and h...,And_CONJ then_ADV whip_VERB the_DET body_NOUN ...,NaN,train-clean-100\LibriSpeech\train-clean-100\91...,whip_anger,hostile


In [118]:
df.to_csv('all_mark_text.csv', index = False)

In [119]:
df['overall_mood'].value_counts()

overall_mood
neutral           7411
peaceful          1188
sunny             1175
pleasant          1125
harmonious        1117
inspiring         1083
restless           967
watchful           957
curious            956
eager              900
hopeful            865
aggressive         663
tense              609
furious            608
stormy             606
hostile            589
depressing         503
dark               496
cold               476
bleak              474
harsh              468
vibrant            347
delightful         341
joyful             335
bright             327
cheerful           325
terrifying         290
eerie              290
paranoid           281
ominous            265
claustrophobic     261
secure             229
calm               225
warm               220
gentle             210
reassuring         196
grim               168
repulsive          166
nauseating         166
disturbing         156
unexpected          74
dramatic            68
abrupt              6

In [110]:
# Расширенный маппинг жанров на обобщённые категории
extended_genre_map = {
    # Fiction
    'Fiction': 'Fiction',
    'Historical fiction': 'Fiction',
    'Psychological fiction': 'Fiction',
    'Political fiction': 'Fiction',
    'Love stories': 'Fiction',
    ' Fiction': 'Fiction',
    'Fiction"': 'Fiction',
    'Western stories': 'Fiction',
    'Utopias': 'Fiction',

    # Fantasy / Sci-Fi
    'Science fiction': 'Fantasy / Sci-Fi',
    'Paranormal fiction': 'Fantasy / Sci-Fi',
    'Fantasy fiction': 'Fantasy / Sci-Fi',
    'Fantasy literature': 'Fantasy / Sci-Fi',
    'Mythology': 'Fantasy / Sci-Fi',
    'Ghost stories': 'Fantasy / Sci-Fi',

    # Mystery / Crime
    'Detective': 'Mystery / Crime',
    'Crime': 'Mystery / Crime',
    'Mystery fiction': 'Mystery / Crime',
    'Detective and mystery stories': 'Mystery / Crime',
    'Corrupt practices': 'Mystery / Crime',

    # Children / Juvenile
    'Juvenile fiction': 'Children / Juvenile',
    "Children's stories": 'Children / Juvenile',
    'Fairy tales': 'Children / Juvenile',
    'Folklore': 'Children / Juvenile',
    "Children's literature": 'Children / Juvenile',

    # Non-fiction
    'Biography': 'Non-fiction',
    'Cooking': 'Non-fiction',
    'Cooking (Fish)': 'Non-fiction',
    'Description and travel': 'Non-fiction',
    'Ethics': 'Non-fiction',
    'Psychology': 'Non-fiction',
    'Mental healing': 'Non-fiction',
    'Inspiration': 'Non-fiction',
    'Philosophy': 'Non-fiction',
    'Doctrines': 'Non-fiction',
    'Essays': 'Non-fiction',
    'Economics': 'Non-fiction',
    'Bible': 'Non-fiction',
    'Encyclopedias and dictionaries': 'Non-fiction',
    'History': 'Non-fiction',
    'Civilization': 'Non-fiction',
    'Marriage': 'Non-fiction',
    'Yosemite National Park (Calif.); Yosemite Valley (Calif.)': 'Non-fiction',
    'Indians of North America': 'Non-fiction',
    'Overland journeys to the Pacific': 'Non-fiction',
    'Voyages around the world': 'Non-fiction',
    'Representative government and representation': 'Non-fiction',
    'Character; Conduct of life': 'Non-fiction',
    'Food supply': 'Non-fiction',
    'Terms and phrases': 'Non-fiction',

    # Poetry / Short Form
    'Poetry': 'Poetry / Short Form',
    'Short stories': 'Poetry / Short Form',
    'Melancholy': 'Poetry / Short Form',
    'Musical notation': 'Poetry / Short Form',
    'Meditations': 'Poetry / Short Form',
    'Humor': 'Poetry / Short Form',

    # Other
    'Motion pictures': 'Other',
    'Music': 'Other',
    'Spiritual life': 'Other',
    'Patterns': 'Other',
    'Physical geography': 'Other',
    'Suffrage': 'Other',
    'Hypnotism': 'Other',
    'Personality': 'Other',
    'Senses and sensation': 'Other',
    'Psychoanalysis': 'Other',
    'Heroes': 'Other',

    # Unknown
    'Not Found': 'Unknown'
}

# Замена значений
data['GENRE'] = data['GENRE'].map(extended_genre_map).fillna('Other')

# Сводка по категориям
data['GENRE'].value_counts()


GENRE
Fiction                270
Children / Juvenile    116
Non-fiction             76
Fantasy / Sci-Fi        63
Poetry / Short Form     23
Mystery / Crime         21
Other                   14
Unknown                  2
Name: count, dtype: int64

In [ ]:
# Укажи корневую папку с BOOK ID-папками
root_dir = "book_illustrations"  # ← замени на свой путь
output_csv = "annotations_template.csv"

image_extensions = {".mp3"}

rows = []

for book_id in os.listdir(root_dir):
    book_path = os.path.join(root_dir, book_id)
    if os.path.isdir(book_path):
        for fname in os.listdir(book_path):
            ext = os.path.splitext(fname)[1].lower()
            if ext in image_extensions:
                image_path = os.path.join(book_path, fname)
                rows.append([book_id, image_path, "", "", "", ""])

# Сохраняем в CSV
with open(output_csv, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["BOOK ID", "filename", "scene", "characters", "style", "attributes"])
    writer.writerows(rows)

print(f"CSV создан: {output_csv} (найдено {len(rows)} изображений)")

CSV создан: annotations_template.csv (найдено 5148 изображений)


In [ ]:
# Жанровые метки
genre_labels = {
    "Fiction": ["calm", "melancholic", "tense", "emotional", "reflective"],
    "Children / Juvenile": ["cheerful", "curious", "playful", "instructive", "carefree"],
    "Non-fiction": ["neutral", "confident", "inspiring", "formal", "thoughtful"],
    "Fantasy / Sci-Fi": ["mysterious", "epic", "tense", "enthusiastic", "surprised"],
    "Poetry / Short Form": ["lyrical", "dreamy", "sad", "solemn", "contemplative"],
    "Mystery / Crime": ["intriguing", "tense", "suspicious", "cold", "shocked"]
}

# Загрузим справочник с жанрами
genre_map = {}
with open("train_with_genre.csv", newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        key = (str(row["READER"]), str(row["ID"]).zfill(4))
        genre_map[key] = {
            "book_id": str(row["BOOK ID"]),
            "genre": row.get("GENRE", "").strip()
        }

# Сохраняем строки по ID
grouped_by_id = defaultdict(list)

# Пройдём по аудиофайлам
for filepath in glob.glob("train-clean-100/LibriSpeech/train-clean-100/*/*/*.wav"):
    parts = filepath.split(os.sep)
    reader = str(parts[-3])
    filename = os.path.basename(filepath)
    id_part = filename.split(".")[0].split("-")[-2].zfill(4)

    key = (reader, id_part)
    book_info = genre_map.get(key)

    if not book_info:
        print(f"Нет BOOK ID или GENRE для {key}")
        continue

    genre = book_info["genre"]
    book_id = book_info["book_id"]

    label_options = genre_labels.get(genre, ["neutral"])
    label = random.choice(label_options)

    grouped_by_id[id_part].append({
        "ID": id_part,
        "READER": reader,
        "BOOK ID": book_id,
        "start": '',
        "end": '',
        "label": label,
        "temp": '',
        "speaker's gender": '',
        "source_file": filepath
    })

# Присваиваем token_id и формируем итог
output = []
for id_part, rows in grouped_by_id.items():
    for i, row in enumerate(rows):
        row["token_id"] = i
        output.append(row)

# Сохраняем CSV
fieldnames = ["ID", "READER", "BOOK ID", "token_id", "start", "end", "label", "temp", "speaker's gender", "source_file"]
with open("all_mark_audio.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(output)


In [147]:
# Загрузка DataFrame
df = pd.read_csv("all_mark_audio.csv")

# Функция для получения длительности файла
def get_duration(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)
        return 0.0, round(librosa.get_duration(y=y, sr=sr), 3)
    except Exception as e:
        print(f"Ошибка при обработке файла {file_path}: {e}")
        return None, None

# Применение функции к каждой строке
starts, ends = [], []
for path in tqdm(df['source_file']):
    normalized_path = os.path.normpath(path)
    start, end = get_duration(normalized_path)
    starts.append(start)
    ends.append(end)

df['start'] = starts
df['end'] = ends

# Сохранение результата
df.to_csv("updated_file.csv", index=False)

100%|██████████| 28539/28539 [03:53<00:00, 122.12it/s]


In [154]:
# Загрузка основного датафрейма с аудио (где нужно заполнить 'words')
df_audio = pd.read_csv("all_mark_audio.csv")

# Загрузка датафрейма с текстами
df_text = pd.read_csv("all_mark_text.csv")

# Убедимся, что token_id — числовой (важно для join)
df_audio['token_id'] = df_audio['token_id'].astype(int)
df_text['token_id'] = df_text['token_id'].astype(int)

# Добавим колонку с числом слов в текстовом df
df_text['text'] = df_text['text'].fillna("").apply(lambda x: len(str(x).split()))

# Объединим по READER, ID и token_id
df_audio = df_audio.merge(
    df_text[['READER', 'ID', 'token_id', 'text']],
    on=['READER', 'ID', 'token_id'],
    how='left'
)
df_audio

,ID,READER,BOOK ID,token_id,start,end,label,temp,speaker's gender,source_file,text
0,1240,103,45,0,0.0,14.085,emotional,NaN,female,train-clean-100/LibriSpeech/train-clean-100\10...,34
1,1240,103,45,1,0.0,15.945,melancholic,NaN,female,train-clean-100/LibriSpeech/train-clean-100\10...,5
2,1240,103,45,2,0.0,13.945,reflective,NaN,female,train-clean-100/LibriSpeech/train-clean-100\10...,19
3,1240,103,45,3,0.0,14.710,tense,NaN,female,train-clean-100/LibriSpeech/train-clean-100\10...,25
4,1240,103,45,4,0.0,12.515,melancholic,NaN,female,train-clean-100/LibriSpeech/train-clean-100\10...,24
...,...,...,...,...,...,...,...,...,...,...,...
28534,130578,911,11938,16,0.0,5.080,cheerful,NaN,male,train-clean-100/LibriSpeech/train-clean-100\91...,15
28535,130578,911,11938,17,0.0,15.165,instructive,NaN,male,train-clean-100/LibriSpeech/train-clean-100\91...,3
28536,130578,911,11938,18,0.0,12.440,carefree,NaN,male,train-clean-100/LibriSpeech/train-clean-100\91...,20
28537,130578,911,11938,19,0.0,14.655,carefree,NaN,male,train-clean-100/LibriSpeech/train-clean-100\91...,15


In [ ]:
# Функция для получения длительности аудио
def get_duration(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)
        return librosa.get_duration(y=y, sr=sr)
    except:
        return None

# Заполнение длительности
durations = []
for path in df_audio['source_file']:
    norm_path = os.path.normpath(path)
    durations.append(get_duration(norm_path))
df_audio['duration'] = durations

# Вычисление скорости речи
df_audio['wpm'] = df_audio['text'] / df_audio['duration']

# Классификация темпа: можно подправить пороги
def classify_temp(wpm):
    if wpm is None:
        return None
    elif wpm > 2.5:
        return "fast"
    elif wpm < 1.5:
        return "slow"
    else:
        return "norm"

df_audio['temp'] = df_audio['wpm'].apply(classify_temp)
df_audio

,ID,READER,BOOK ID,token_id,start,end,label,temp,speaker's gender,source_file,text,duration,wpm
0,1240,103,45,0,0.0,14.085,emotional,norm,female,train-clean-100/LibriSpeech/train-clean-100\10...,34,14.085,2.413916
1,1240,103,45,1,0.0,15.945,melancholic,slow,female,train-clean-100/LibriSpeech/train-clean-100\10...,5,15.945,0.313578
2,1240,103,45,2,0.0,13.945,reflective,slow,female,train-clean-100/LibriSpeech/train-clean-100\10...,19,13.945,1.362496
3,1240,103,45,3,0.0,14.710,tense,norm,female,train-clean-100/LibriSpeech/train-clean-100\10...,25,14.710,1.699524
4,1240,103,45,4,0.0,12.515,melancholic,norm,female,train-clean-100/LibriSpeech/train-clean-100\10...,24,12.515,1.917699
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28534,130578,911,11938,16,0.0,5.080,cheerful,fast,male,train-clean-100/LibriSpeech/train-clean-100\91...,15,5.080,2.952756
28535,130578,911,11938,17,0.0,15.165,instructive,slow,male,train-clean-100/LibriSpeech/train-clean-100\91...,3,15.165,0.197824
28536,130578,911,11938,18,0.0,12.440,carefree,norm,male,train-clean-100/LibriSpeech/train-clean-100\91...,20,12.440,1.607717
28537,130578,911,11938,19,0.0,14.655,carefree,slow,male,train-clean-100/LibriSpeech/train-clean-100\91...,15,14.655,1.023541


In [158]:
df_audio = df_audio.drop(columns = ['text', 'duration'])
df_audio

,ID,READER,BOOK ID,token_id,start,end,label,temp,speaker's gender,source_file,wpm
0,1240,103,45,0,0.0,14.085,emotional,norm,female,train-clean-100/LibriSpeech/train-clean-100\10...,2.413916
1,1240,103,45,1,0.0,15.945,melancholic,slow,female,train-clean-100/LibriSpeech/train-clean-100\10...,0.313578
2,1240,103,45,2,0.0,13.945,reflective,slow,female,train-clean-100/LibriSpeech/train-clean-100\10...,1.362496
3,1240,103,45,3,0.0,14.710,tense,norm,female,train-clean-100/LibriSpeech/train-clean-100\10...,1.699524
4,1240,103,45,4,0.0,12.515,melancholic,norm,female,train-clean-100/LibriSpeech/train-clean-100\10...,1.917699
...,...,...,...,...,...,...,...,...,...,...,...
28534,130578,911,11938,16,0.0,5.080,cheerful,fast,male,train-clean-100/LibriSpeech/train-clean-100\91...,2.952756
28535,130578,911,11938,17,0.0,15.165,instructive,slow,male,train-clean-100/LibriSpeech/train-clean-100\91...,0.197824
28536,130578,911,11938,18,0.0,12.440,carefree,norm,male,train-clean-100/LibriSpeech/train-clean-100\91...,1.607717
28537,130578,911,11938,19,0.0,14.655,carefree,slow,male,train-clean-100/LibriSpeech/train-clean-100\91...,1.023541


In [160]:
# Указание нового порядка колонок
cols = df_audio.columns.tolist()
# Удаляем 'wpm' из текущего положения
cols.remove('wpm')
# Вставляем 'wpm' после 'temp'
temp_index = cols.index('temp') + 1
cols.insert(temp_index, 'wpm')

# Переупорядочиваем DataFrame
df_audio = df_audio[cols]
df_audio

,ID,READER,BOOK ID,token_id,start,end,label,temp,wpm,speaker's gender,source_file
0,1240,103,45,0,0.0,14.085,emotional,norm,2.413916,female,train-clean-100/LibriSpeech/train-clean-100\10...
1,1240,103,45,1,0.0,15.945,melancholic,slow,0.313578,female,train-clean-100/LibriSpeech/train-clean-100\10...
2,1240,103,45,2,0.0,13.945,reflective,slow,1.362496,female,train-clean-100/LibriSpeech/train-clean-100\10...
3,1240,103,45,3,0.0,14.710,tense,norm,1.699524,female,train-clean-100/LibriSpeech/train-clean-100\10...
4,1240,103,45,4,0.0,12.515,melancholic,norm,1.917699,female,train-clean-100/LibriSpeech/train-clean-100\10...
...,...,...,...,...,...,...,...,...,...,...,...
28534,130578,911,11938,16,0.0,5.080,cheerful,fast,2.952756,male,train-clean-100/LibriSpeech/train-clean-100\91...
28535,130578,911,11938,17,0.0,15.165,instructive,slow,0.197824,male,train-clean-100/LibriSpeech/train-clean-100\91...
28536,130578,911,11938,18,0.0,12.440,carefree,norm,1.607717,male,train-clean-100/LibriSpeech/train-clean-100\91...
28537,130578,911,11938,19,0.0,14.655,carefree,slow,1.023541,male,train-clean-100/LibriSpeech/train-clean-100\91...


In [161]:
df_audio.to_csv("all_mark_audio.csv", index=False)

# Просмотр полученных данных

In [ ]:
df = pd.read_csv('all_mark_image.csv')
df.head(7)

,BOOK ID,image,scene,characters,style,attributes
0,16,book_illustrations\16\cover.jpg,драматичная сцена сражения на пиратском корабле,"Питер Пэн, капитан Крюк",классический стиль,"сепия, экшн, штамп библиотеки"
1,45,book_illustrations\45\cover.jpg,девочка стоит на мосту посреди весеннего пейзажа,девочка,сказочный стиль,"цветы, яркие краски, стилизованные деревья"
2,47,book_illustrations\47\cover.jpg,портрет женщины на коричневой обложке,женщина,"реалистичная живопись, винтаж","приглушённые цвета, формальный стиль, начало X..."
3,59,NaN,NaN,NaN,NaN,NaN
4,60,book_illustrations\60\cover.jpg,портрет мужчины и женщины,"женщина, мужчина","реалистичная живопись, темные цвета","трость, шляпа, аристократская одежда"
...,...,...,...,...,...,...
5143,43699,book_illustrations\43699\logo.jpg,NaN,NaN,NaN,NaN
5144,44326,book_illustrations\44326\colophon.png,NaN,NaN,NaN,NaN
5145,44326,book_illustrations\44326\cover.jpg,NaN,NaN,NaN,NaN
5146,1000006,book_illustrations\1000006\cover.jpg,обложка,NaN,минимализм,"нарисованный человечек, темный прямоугольник"


In [2]:
df = pd.read_csv('all_mark_text.csv')
df#.head(7)

,READER,ID,BOOK ID,token_id,text,POS_tags,NER_tags,source_file,emotion_text,overall_mood
0,19,198,121,0,Northanger Abbey,Northanger_NOUN Abbey_NOUN,NaN,train-clean-100\LibriSpeech\train-clean-100\19...,NaN,neutral
1,19,198,121,1,This little work was finished in the year eigh...,This_DET little_ADJ work_NOUN was_VERB finishe...,the year eighteen_DATE three_CARDINAL,train-clean-100\LibriSpeech\train-clean-100\19...,intended_anticipation,restless
2,19,198,121,2,Neither the author nor the public have any oth...,Neither_CONJ the_DET author_NOUN nor_CONJ the_...,thirteen years_DATE,train-clean-100\LibriSpeech\train-clean-100\19...,author_positive public_anticipation,pleasant
3,19,198,121,3,The public are entreated to bear in mind that ...,The_DET public_NOUN are_VERB entreated_VERB to...,thirteen years_DATE,train-clean-100\LibriSpeech\train-clean-100\19...,public_anticipation bear_anger begun_anticipation,restless
4,19,198,121,4,Chapter One No one who had ever seen Catherine...,Chapter_NOUN One_NUM No_NOUN one_NOUN who_PRON...,Catherine Morland_PERSON,train-clean-100\LibriSpeech\train-clean-100\19...,heroine_positive,harmonious
...,...,...,...,...,...,...,...,...,...,...
28534,8630,305213,35941,50,Until the truth was fully discussed and proved,Until_ADP the_DET truth_NOUN was_VERB fully_AD...,NaN,train-clean-100\LibriSpeech\train-clean-100\86...,truth_positive fully_positive,peaceful
28535,8630,305213,35941,51,That was what the Manicheans promised what Aug...,That_DET was_VERB what_PRON the_DET Manicheans...,Manicheans_NORP Augustine_PERSON,train-clean-100\LibriSpeech\train-clean-100\86...,found_joy,delightful
28536,8630,305213,35941,52,They taught what was false not only about thee...,They_PRON taught_VERB what_PRON was_VERB false...,NaN,train-clean-100\LibriSpeech\train-clean-100\86...,taught_trust god_anticipation art_anticipation...,restless
28537,8630,305213,35941,53,He found them carnal and loquacious full of in...,He_PRON found_VERB them_PRON carnal_ADJ and_CO...,NaN,train-clean-100\LibriSpeech\train-clean-100\86...,found_joy carnal_negative full_positive insane...,delightful


In [3]:
df = pd.read_csv('all_mark_audio.csv')
df#.head(7)

,ID,READER,BOOK ID,token_id,start,end,label,temp,wpm,speaker's gender,source_file
0,198,19,121,0,0.0,1.965,tense,slow,1.017812,female,train-clean-100/LibriSpeech/train-clean-100\19...
1,198,19,121,1,0.0,11.765,tense,slow,1.359966,female,train-clean-100/LibriSpeech/train-clean-100\19...
2,198,19,121,2,0.0,10.795,reflective,fast,2.686429,female,train-clean-100/LibriSpeech/train-clean-100\19...
3,198,19,121,3,0.0,15.480,melancholic,slow,1.485788,female,train-clean-100/LibriSpeech/train-clean-100\19...
4,198,19,121,4,0.0,12.990,calm,norm,1.693610,female,train-clean-100/LibriSpeech/train-clean-100\19...
...,...,...,...,...,...,...,...,...,...,...,...
28534,305213,8630,35941,50,0.0,14.635,reflective,slow,0.546635,male,train-clean-100/LibriSpeech/train-clean-100\86...
28535,305213,8630,35941,51,0.0,14.265,emotional,slow,1.051525,male,train-clean-100/LibriSpeech/train-clean-100\86...
28536,305213,8630,35941,52,0.0,15.170,calm,norm,1.713909,male,train-clean-100/LibriSpeech/train-clean-100\86...
28537,305213,8630,35941,53,0.0,16.170,emotional,slow,0.618429,male,train-clean-100/LibriSpeech/train-clean-100\86...


In [5]:
df = pd.read_csv('all_book.csv')
df

,ID,READER,MINUTES,SUBSET,PROJ.,BOOK ID,CH. TITLE,PROJECT TITLE,GENRE
0,198,19,8.42,train-clean-100,219,121,Chapter 01,Northanger Abbey,Fiction
1,209,198,8.20,train-clean-100,219,121,Chapter 12,Northanger Abbey,Fiction
2,216,403,6.30,train-clean-100,219,121,Chapter 19,Northanger Abbey,Fiction
3,218,89,12.78,train-clean-100,219,121,Chapter 21,Northanger Abbey,Fiction
4,219,89,12.30,train-clean-100,219,121,Chapter 22,Northanger Abbey,Fiction
...,...,...,...,...,...,...,...,...,...
580,298545,8838,12.55,train-clean-100,8386,10446,"The Croxley Master - I, part 1",Green Flag and Other Stories of War and Sport,Fiction
581,298546,8838,12.51,train-clean-100,8386,10446,"The Croxley Master - I, part 2",Green Flag and Other Stories of War and Sport,Fiction
582,300880,6064,8.82,train-clean-100,8473,44326,Chapter 10,Wanted: A Husband,Fiction
583,305212,8630,10.50,train-clean-100,8645,35941,How Augustine Went to Carthage and How Patrici...,Life of Saint Monica,Fiction
